In [ ]:
import os

from ase.io import write as ase_write
from ase.optimize import BFGS
from ase import Atoms

from pymatgen.ext.matproj import MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.ase import AseAtomsAdaptor


In [ ]:
import os
env_var = "MP_API_KEY"
mp_api_key = os.environ.get(env_var, None)

In [ ]:
mp_id="mp-352"
with MPRester(mp_api_key) as m:
    struct = m.get_structure_by_material_id(mp_id)

In [ ]:
sga = SpacegroupAnalyzer(struct, symprec=1e-4)
conv = sga.get_conventional_standard_structure()

conv_atoms = AseAtomsAdaptor.get_atoms(conv)

In [ ]:
conv_atoms

In [ ]:
ase_write("ref_HfO2.xyz", conv_atoms, format="extxyz")

In [ ]:
import matplotlib.pyplot as plt
from ase.visualize.plot import plot_atoms

fig, ax = plt.subplots()
plot_atoms(conv_atoms, ax, radii=0.3, rotation=('45x,45y,90z'))
ax.set_title(conv_atoms.get_chemical_formula())
ax.set_axis_off()
plt.show()

In [ ]:
ase_write("mp_conv_monoclinic_HfO2.xyz", conv_atoms, format="extxyz")

In [ ]:
from mace.calculators import MACECalculator
mace_calc = MACECalculator(model_paths="../../models/HfOx_128x0e_100725_1007_from-scratch_stagetwo.model", device="cpu")

In [ ]:
conv_atoms.set_calculator(mace_calc)

In [ ]:
from ase.filters import ExpCellFilter
from ase.io.trajectory import Trajectory

ecf = ExpCellFilter(conv_atoms, hydrostatic_strain=True)
opt = BFGS(ecf, logfile="relax.txt")
traj = Trajectory("conv_monoclinic_HfO2_relax.traj", "w", conv_atoms)
opt.attach(traj)
opt.run(fmax=0.05)

In [ ]:
from ase.io import read as ase_read
to_write_traj = traj = ase_read("conv_monoclinic_HfO2_relax.traj", index=":")
ase_write("relaxation.xyz", to_write_traj)

In [ ]:
import numpy as np

cell = conv_atoms.cell.array
conv_atoms.set_cell(np.round(cell,decimals=15))

What it was:
{'array': array([[-2.15712578e-17,  5.15141070e+00,  0.00000000e+00],
        [ 5.20851486e+00, -2.68855741e-19,  0.00000000e+00],
        [ 8.30070492e-19, -8.86013196e-01, -5.26141137e+00]])}

In [ ]:
conv_atoms.cell.array

In [ ]:
len(conv_atoms)

In [ ]:
HfO2_555 = conv_atoms.repeat((5,5,5))

In [ ]:
HfO2_222 = conv_atoms.repeat((2,2,2))

In [ ]:
ase_write("HfO2_222_MACE100725_DATA", HfO2_222, format="lammps-data", masses=True)

In [ ]:
fig, ax = plt.subplots()
plot_atoms(HfO2_555, ax, radii=0.3, rotation=('45x,45y,90z'))
ax.set_title(HfO2_555.get_chemical_formula())
ax.set_axis_off()
plt.show()

In [ ]:
HfO2_101010 = conv_atoms.repeat((10,10,10))

In [ ]:
len(HfO2_101010)

In [ ]:
HfO2_212121 = conv_atoms.repeat((21,21,21))

In [ ]:
ase_write("HfO2_555_MACE100725_DATA", HfO2_555, format="lammps-data", masses=True)

In [ ]:
ase_write("HfO2_101010_MACE100725_DATA", HfO2_101010, format="lammps-data", masses=True)
ase_write("HfO2_212121_MACE100725_DATA", HfO2_212121, format="lammps-data", masses=True)